# AWS Chatbot Custom Alerting

This notebook demonstrates how to send custom notifications to AWS Chatbot using the `ChatbotNotifier` client.

**SNS Topic ARN:** `arn:aws:sns:eu-west-1:760097843905:testing-alerting-chatbot-events`

**Note:** Only basic markdown is supported in notifications. Complex markdown features like tables, footnotes, and some formatting may not render correctly. See the [Slack markdown guide](https://www.markdownguide.org/tools/slack/) for supported features.

In [ ]:
# Configure AWS profile for local development
%env AWS_PROFILE=platform-developer

In [ ]:
import sys
import os
import uuid

# Add src to path if not present
src_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'src'))
if src_path not in sys.path:
    sys.path.append(src_path)

import boto3
from clients.chatbot_notifier import ChatbotNotifier, ChatbotMessage

# Configure SNS client
sns_client = boto3.client("sns", region_name="eu-west-1")
topic_arn = "arn:aws:sns:eu-west-1:760097843905:testing-alerting-chatbot-events"

# Create the notifier
notifier = ChatbotNotifier(sns_client=sns_client, topic_arn=topic_arn)

print("✅ ChatbotNotifier initialized successfully")

## Send a Simple Message

Send a basic markdown-formatted message to the chatbot.

In [ ]:
# Generate a unique thread ID for this conversation
thread_id = str(uuid.uuid4())
print(f"🧵 Thread ID: {thread_id}")

# Send a simple message
message = ChatbotMessage(
    text="Hello from Catalogue Pipeline! 👋\n\n *Status:* Testing custom notifications\n\n- Feature: ChatbotNotifier\n- Date: 2025-12-02",
    thread_id=thread_id
)

response = notifier.send_notification(message)
print(f"✅ Message sent successfully!")
print(f"📧 Message ID: {response.message_id}")

## Send a Threaded Reply

Send a follow-up message in the same thread using the message ID from the previous response.

In [ ]:
# Send a follow-up message in the same thread
follow_up = ChatbotMessage(
    text="✅ *Update:* All tests passing!\n\nThe ChatbotNotifier client is working as expected.",
    thread_id=thread_id  # Use the same thread ID
)

follow_up_response = notifier.send_notification(follow_up)
print(f"✅ Follow-up message sent in thread!")
print(f"📧 New Message ID: {follow_up_response.message_id}")

## Send a Message with Metadata

Include additional context with metadata fields.

In [ ]:
# Send a message with metadata
from datetime import datetime, timezone

message_with_metadata = ChatbotMessage(
    text="Axiell Adapter Status Report\n\n*Job ID:* 202512021430\n\n*Windows Processed:* 24/50\n\n*Status:* In Progress ⏳",
    thread_id=thread_id,  # Keep it in the same thread
    metadata={
        "adapter": "axiell",
        "job_id": "202512021430",
        "timestamp": datetime.now(timezone.utc).isoformat(),
        "status": "in_progress"
    }
)

metadata_response = notifier.send_notification(message_with_metadata)
print(f"✅ Message with metadata sent!")
print(f"📧 Message ID: {metadata_response.message_id}")